In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup

# 카카오맵 URL
URL = 'https://map.kakao.com/'
KEYWORD = '가산동 맛집'

# Chrome WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')

driver = webdriver.Chrome(options=options)
driver.get(URL)

# 검색어 입력 및 검색 실행
search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]')
search_area.send_keys(KEYWORD)
driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)
time.sleep(3)

# 장소 탭 클릭
driver.find_element(By.XPATH, '//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)
time.sleep(3)

# 검색 결과 HTML 파싱
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
store_names = [store.text.strip() for store in soup.select('.head_item > .tit_name > .link_name')]



print("검색된 가게 목록:")
for name in store_names:
    print(name)

# 가게 상세 페이지 크롤링
for store_name in store_names:
    try:
        name_element = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.LINK_TEXT, store_name)))
        ActionChains(driver).move_to_element(name_element).click().perform()
        time.sleep(2)
        
        # 상세 페이지 링크 가져오기
        place_links = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[data-id="name"]')))
        for link in place_links:
            href = link.get_attribute("href")
            if href and "none" not in href:
                print(f"{store_name} 링크: {href}")
                driver.execute_script(f"window.open('{href}');")
                time.sleep(1)
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(1)

                # 상세 페이지 크롤링
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')

                st_name = soup.find('h3', class_='tit_place')
                st_name = st_name.get_text(strip=True).replace('장소명', '') if st_name else '이름 없음'
                
                rating = soup.find('span', class_='num_star')
                rating = rating.get_text(strip=True) if rating else '평점 없음'
                
                reviews = soup.find('a', class_='link_review')
                reviews = reviews.find('span', class_='info_num').get_text(strip=True) if reviews else '후기 없음'
                
                blog_reviews = soup.find('a', class_='link_blog')
                blog_reviews = blog_reviews.find('span', class_='info_num').get_text(strip=True) if blog_reviews else '블로그 후기 없음'
                
                # 장소 카테고리 크롤링
                category_element = soup.find('span', class_='info_cate')
                category = category_element.get_text(strip=True).replace('장소 카테고리', '') if category_element else '카테고리 없음'
                
                # 메뉴 탭 클릭 (다양한 구조 대응)
                try:
                    menu_tab = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "li:nth-child(3) > a.link_tab"))
                    )
                except:
                    menu_tab = driver.find_element(By.CSS_SELECTOR, "a.link_tab[role='tab']")
                
                if menu_tab:
                    menu_tab.click()
                    time.sleep(2)
                else:
                    print(f"{store_name} 메뉴 탭 없음")

                # 메뉴 정보 크롤링
                menu_items = soup.find_all('div', class_='info_goods')
                menu_count = len(menu_items)
                if menu_count >= 3:
                    menu_items = menu_items[:3]  # 메뉴가 3개 이상이면 상위 3개만 가져옴
                
                menus = [(item.find('strong', class_='tit_item').get_text(strip=True), item.find('p', class_='desc_item').get_text(strip=True)) for item in menu_items] if menu_items else []

                print(f'장소명: {st_name}')
                print(f'별점: {rating}')
                print(f'후기 개수: {reviews}')
                print(f'블로그 후기 개수: {blog_reviews}')
                print(f'카테고리: {category}')
                print('메뉴 목록:')
                if menus:
                    for menu in menus:
                        print(f'- {menu[0]}: {menu[1]}')
                else:
                    print('메뉴 없음')

                # 창 닫고 원래 창으로 복귀
                driver.close()
                time.sleep(1)
                driver.switch_to.window(driver.window_handles[0])
                break
    except Exception as e:
        print(f"{store_name} 클릭 실패: {e}")

# 드라이버 종료
driver.quit()

검색된 가게 목록:
춘천옥
삼숙이갈비 본점
가산어시장
스시쥬베이
북촌삼대갈비 가산본점
도원
인크커피 가산점
개성손만두요리전문점 가산디지털점
동산미
가산물갈비&즉석소불고기 - 별점 없음, 스킵
맥도날드 가산비지니스센터점
호세가
풍년갈비
동남집
통영해물가 가산2호점
장소명: 이름 없음
별점: 평점 없음
후기 개수: 후기 없음
블로그 후기 개수: 블로그 후기 없음
카테고리: 카테고리 없음
주소: 주소 없음
춘천옥 클릭 실패: Message: invalid session id
Stacktrace:
#0 0x5873c4873ffa <unknown>
#1 0x5873c43327c3 <unknown>
#2 0x5873c43755df <unknown>
#3 0x5873c43aa196 <unknown>
#4 0x5873c43a4813 <unknown>
#5 0x5873c43a3ae5 <unknown>
#6 0x5873c42fb528 <unknown>
#7 0x5873c483b42b <unknown>
#8 0x5873c483f2ec <unknown>
#9 0x5873c4822a22 <unknown>
#10 0x5873c483fe64 <unknown>
#11 0x5873c4806bef <unknown>
#12 0x5873c42f9f5b <unknown>
#13 0x7c0179e29d90 <unknown>

삼숙이갈비 본점 클릭 실패: Message: invalid session id
Stacktrace:
#0 0x5873c4873ffa <unknown>
#1 0x5873c43327c3 <unknown>
#2 0x5873c43755df <unknown>
#3 0x5873c43aa196 <unknown>
#4 0x5873c43a4813 <unknown>
#5 0x5873c43a3ae5 <unknown>
#6 0x5873c42fb528 <unknown>
#7 0x5873c483b42b <unknown>
#8 0x5873c483f2ec <unknown>
#9 0x5873c4822a22 